# Hybrid Search vs. Similarity Search

In diesem Notebook lernst du den Unterschied zwischen **Hybrid Search** und **Similarity Search** kennen anhand eines realen Beispiels: **andrena objects** (Softwarefirma) wird oft mit **Andrena** (Wildbienengattung) verwechselt!

## Das Problem: Homonyme und Mehrdeutigkeit

**andrena objects** ist eine Softwareentwicklungsfirma aus Karlsruhe, die sich auf agile Methoden spezialisiert hat. Ihr Logo enthält eine Biene, denn "Andrena" ist auch der wissenschaftliche Name für Sandbienen (eine Wildbienengattung).

Diese Doppeldeutigkeit führt zu interessanten Herausforderungen bei der Suche:
- Suche nach "andrena" könnte beide Bedeutungen zurückgeben
- Suche nach "andrena objects" sollte eindeutig die Firma finden
- Suche nach "Firma mit Biene" erfordert semantisches Verständnis



## Der Demo-Datensatz

Wir erstellen einen Beispieldatensatz mit drei Kategorien von Dokumenten:

1. **andrena objects (Firma)** - Dokumente über die Softwarefirma
2. **Andrena (Wildbienen)** - Dokumente über die Bienengattung
3. **Andere Themen** - Rauschen (andere Firmen, andere Insekten, IT-Themen)

Dieser Mix simuliert eine realistische Wissensdatenbank, in der beide Bedeutungen vorkommen können.

In [ ]:
# Beispieldokumente - achte auf die Doppeldeutigkeit von "andrena"!
beispiel_dokumente = [
    # andrena objects - Softwarefirma (6 Dokumente)
    "andrena objects ist eine Softwareentwicklungsfirma mit Sitz in Karlsruhe, die sich auf agile Entwicklungsmethoden und Clean Code spezialisiert hat.",
    "andrena objects bietet Dienstleistungen in den Bereichen Software-Entwicklung, Consulting, Coaching und Training für agile Methoden an.",
    "Das Team von andrena objects besteht aus erfahrenen Software-Entwicklern, die Wert auf Qualität und handwerkliches Können legen.",
    "andrena objects setzt auf moderne Technologien wie Java, Python, TypeScript und Cloud-native Architekturen für Kundenprojekte.",
    "Die Unternehmenskultur bei andrena objects fördert kontinuierliches Lernen, Pair Programming und Test-Driven Development.",
    
    # Andrena - Wildbienen (6 Dokumente)
    "Andrena ist eine Gattung von Wildbienen, die auch als Sandbienen oder Erdbienen bekannt sind und weltweit verbreitet vorkommen.",
    "Sandbienen der Gattung Andrena nisten in selbstgegrabenen Röhren im Boden und sind wichtige Bestäuber für Obstbäume und Wildpflanzen.",
    "Die Andrena-Arten sind durch ihre pelzige Behaarung und die Fähigkeit gekennzeichnet, Pollen in speziellen Haarbürsten zu sammeln.",
    "In Deutschland gibt es über 100 verschiedene Andrena-Arten, die von März bis September aktiv sind und bevorzugt sandige Böden besiedeln.",
    "Andrena-Weibchen legen Brutzellen an, in denen sie Pollen und Nektar für ihre Nachkommen sammeln, bevor sie ein Ei ablegen.",
    "Die meisten Andrena-Arten sind oligolektisch, das heißt sie sammeln Pollen nur von bestimmten Pflanzenfamilien wie Weiden oder Kreuzblütlern.",
    
    # Andere Softwarefirmen (5 Dokumente)
    "ThoughtWorks ist eine globale Softwareberatung, die sich auf agile Transformation und maßgeschneiderte Software-Lösungen konzentriert.",
    "Netlight Consulting bietet IT-Beratung und Software-Entwicklung mit Fokus auf digitale Transformation und Innovation in Skandinavien und Europa.",
    "INNOQ ist eine deutsche IT-Beratungsfirma, die sich auf Architektur-Beratung, Entwicklung und Technologie-Strategie spezialisiert hat.",
    "it-agile aus Hamburg bietet Schulungen, Coaching und Beratung für Scrum, Kanban und andere agile Frameworks an.",
    "Zühlke Engineering ist ein internationales Dienstleistungsunternehmen für Innovation und Software-Entwicklung mit Schweizer Wurzeln.",
    
    # Andere Insekten (4 Dokumente)
    "Honigbienen der Art Apis mellifera leben in Staaten mit bis zu 50000 Individuen und produzieren Honig als Wintervorrat.",
    "Hummeln gehören zur Gattung Bombus und sind robuste Bestäuber, die auch bei kühlen Temperaturen fliegen können.",
    "Solitäre Wespen wie die Grabwespen graben Nester im Boden und versorgen ihre Larven mit gelähmten Insekten als Nahrung.",
    "Schwebfliegen imitieren das Aussehen von Bienen und Wespen, sind aber harmlose Blütenbesucher und wichtige Bestäuber.",
    
    # IT-Themen allgemein (5 Dokumente)
    "Test-Driven Development ist eine Entwicklungsmethode, bei der Tests vor dem eigentlichen Code geschrieben werden.",
    "Microservices-Architekturen zerlegen Anwendungen in kleine, unabhängig deploybare Services mit klaren Schnittstellen.",
    "Domain-Driven Design ist ein Ansatz zur Modellierung komplexer Software-Systeme basierend auf der Fachdomäne.",
    "Continuous Integration und Continuous Deployment automatisieren den Build- und Deployment-Prozess für schnellere Releases.",
    "Pair Programming ist eine agile Praxis, bei der zwei Entwickler gemeinsam an einem Computer arbeiten und Code schreiben."
]

## Hands-on: Vergleich der Retrieval-Methoden

Starten wir zunächst mit einem klassischen Vektorretriever

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
import os
import chromadb
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document

from dotenv import load_dotenv

# Lade Umgebungsvariablen
load_dotenv()

# Erstelle Document-Objekte aus den Beispieltexten
documents = [Document(page_content=text) for text in beispiel_dokumente]

embedding_model = os.getenv("EMBEDDING_MODEL", "text-embedding-3-small")
embeddings = OpenAIEmbeddings(model=embedding_model)

# Erstelle einen neuen ephemeral client, um Duplikate bei mehrfachem Ausführen zu vermeiden
client = chromadb.EphemeralClient()
try:
    client.delete_collection("hybrid_search_demo")
except Exception:
    pass  # Collection doesn't exist yet, which is fine

# Erstelle einen temporären Chroma Vector Store
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="hybrid_search_demo",
    client=client
)

vector_retriever = vector_store.as_retriever(search_kwargs={"k": 7})

print("Vector Store Retriever initialisiert")

Wenn wir diesem Retriever nun unsere Frage `"Was ist andrena objects?"` mitgeben erhalten wir folgendes Ergebnis:

In [ ]:
#TODO rufe den vector_retriever mit der frage "Was ist andrena objects?" auf und lasse die die ergebnisse ausgeben


Kein schlechtes Ergebnis, aber einige Dokumente sind für unseren Fall nicht relevant da sie mit den Wildbienen zu tun haben.

Hier kann ein BM25 Retriever abhilfe schaffen. Dieser beachtet nicht wie die Vektorsuche semantische Ähnlichkeit, sondern sucht nach exakten Keyword-Übereinstimmungen. BM25 (Best Matching 25) ist ein Ranking-Algorithmus, der auf Term Frequency (wie oft kommt ein Wort im Dokument vor) und Inverse Document Frequency (wie selten ist ein Wort in allen Dokumenten) basiert. Das bedeutet: Ein Dokument, das beide Begriffe "andrena" **und** "objects" enthält, wird höher gerankt als eines, das nur "andrena" enthält.

In [ ]:
# 1. BM25 Retriever (Keyword-basiert)
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 7 

print("BM25 Retriever initialisiert")

In [ ]:
#TODO rufe den bm25_retriever mit der frage "Was ist andrena objects?" auf und lasse die die ergebnisse ausgeben


Der BM25 Retriever findet keine Dokumente zu den Bienen mehr! **Warum?** Weil die Query "Was ist andrena objects?" beide Begriffe enthält: "andrena" **und** "objects". BM25 priorisiert Dokumente, die beide Keywords enthalten. Die Bienen-Dokumente enthalten zwar "Andrena", aber nie das Wort "objects" - daher werden sie im Ranking nach unten geschoben. Die Firmen-Dokumente mit "andrena objects" bekommen die höchsten Scores.

**Was findet der Hybrid Retriever?** Er kombiniert die Ergebnisse von BM25 (keyword-basiert) und Vector Search (semantisch) mit einer gewichteten Kombination. Bei der Query "Was ist andrena objects?" profitieren wir von beiden Welten:
- **BM25** findet alle Firmen-Dokumente mit dem exakten Begriff "andrena objects" 
- **Vector Search** ergänzt semantisch ähnliche Dokumente
- Das Ergebnis ist eine kombinierte, neu gewichtete Liste, die die Stärken beider Methoden nutzt

Der Hybrid-Ansatz ist besonders robust: Er findet sowohl bei exakten Keyword-Matches (z.B. "andrena objects") als auch bei umschreibenden Fragen (z.B. "Softwarefirma mit Biene") die richtigen Dokumente.

In [ ]:
# 3. Hybrid Retriever (Ensemble)
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]  # 50/50 Gewichtung
)
print("Hybrid Retriever initialisiert")

results = hybrid_retriever.invoke("Was ist andrena objects?")
for i, doc in enumerate(results, 1):
    print(f"  {i}. {doc.page_content}")

### Hilfsfunktion für übersichtliche Ausgabe

In [ ]:
def compare_retrievers(query, retrievers_dict):
    """
    Vergleicht verschiedene Retriever für eine gegebene Query
    
    Args:
        query: Die Suchanfrage
        retrievers_dict: Dict mit Namen und Retriever-Objekten
    """
    print(f"\n{'='*80}")
    print(f"QUERY: {query}")
    print(f"{'='*80}\n")
    
    for name, retriever in retrievers_dict.items():
        print(f"--- {name} ---")
        results = retriever.invoke(query)
        
        for i, doc in enumerate(results, 1):
            print(f"  {i}. {doc.page_content}")
        
        print()

## Test 1: Exakte Firmen-Frage

Frage: **"Was ist andrena objects?"**

**Erwartung:** BM25 sollte die Firma-Dokumente mit dem exakten Begriff "andrena objects" finden und priorisieren.

In [ ]:
compare_retrievers(
    "Was ist andrena objects?",
    {
        "BM25 (Keyword)": bm25_retriever,
        "Vector (Semantic)": vector_retriever,
        "Hybrid (Both)": hybrid_retriever
    }
)

## Test 2: Mehrdeutige Frage

Frage: **"Was macht andrena?"**

**Erwartung:** Diese Frage ist mehrdeutig! Sie könnte sich auf die Firma ODER die Bienengattung beziehen. 
Interessant zu sehen, welche Dokumente die verschiedenen Retriever zurückgeben.

In [ ]:
compare_retrievers(
    "Was macht andrena?",
    {
        "BM25 (Keyword)": bm25_retriever,
        "Vector (Semantic)": vector_retriever,
        "Hybrid (Both)": hybrid_retriever
    }
)

## Test 3: Semantische Firmen-Frage

Frage: **"Erzähle mir über die Softwarefirma mit der Biene im Logo"**

**Erwartung:** Diese Frage erwähnt "andrena" nicht direkt, beschreibt aber semantisch die Firma. Vector Search sollte hier glänzen!

In [ ]:
compare_retrievers(
    "Erzähle mir über die Softwarefirma mit der Biene",
    {
        "BM25 (Keyword)": bm25_retriever,
        "Vector (Semantic)": vector_retriever,
        "Hybrid (Both)": hybrid_retriever
    }
)

## Test 4: Bienen-spezifische Frage

Frage: **"Welche Wildbienen graben Nester im Boden?"**

**Erwartung:** Sollte Andrena-Bienen-Dokumente finden, nicht die Firma!

In [ ]:
compare_retrievers(
    "Welche Wildbienen graben Nester im Boden?",
    {
        "BM25 (Keyword)": bm25_retriever,
        "Vector (Semantic)": vector_retriever,
        "Hybrid (Both)": hybrid_retriever
    }
)

## Was ist der Unterschied zwischen den Such-Methoden?

### Similarity Search (Vector Search)
Verwendet Embeddings für semantische Ähnlichkeit. Versteht Paraphrasen und Synonyme ("Softwarefirma" = "IT-Unternehmen"), aber kann bei mehrdeutigen Begriffen wie "andrena" beide Bedeutungen ähnlich bewerten.

### BM25 (Keyword Search)
Basiert auf exakter Wortübereinstimmung und TF-IDF-Ranking. Exzellent für präzise Begriffe ("andrena objects"), aber versteht keine Synonyme oder semantischen Kontext.

### Hybrid Search (Ensemble Retriever)
Kombiniert beide Methoden mit gewichteter Kombination (50/50):
1. BM25 holt Top-K Dokumente (Keywords)
2. Vector Search holt Top-K Dokumente (Semantik)
3. Beide Listen werden kombiniert und neu gewichtet

**Ideal für "andrena objects" Use Case:**
- "andrena objects" (exakter Name) → BM25
- "Firma mit Biene" (Beschreibung) → Vector Search
- Hybrid nutzt beide Stärken

## Integration in die RAG-Anwendung

Nachdem du die Unterschiede in diesem vereinfachten Beispiel gesehen hast, kannst du die verschiedenen Retriever in der echten Anwendung testen und evaluieren.

### Schritt 1: `.env` Datei anpassen

Öffne die `.env` Datei im Projekt-Root und ändere die folgende Variable:

```bash
# Für Similarity Search (nur Vector)
RETRIEVER_TYPE=SIMILARITY_SEARCH

# Für Hybrid Search (BM25 + Vector)
RETRIEVER_TYPE=HYBRID_SEARCH
```

**Wichtig:** Die Anzahl der abgerufenen Dokumente wird bei Hybrid Search auf beide Retriever aufgeteilt:
- Wenn `RETRIEVED_NUMBER_OF_DOCUMENTS=4`, dann holt jeder Retriever 2 Dokumente
- Die finale Liste enthält maximal 4 Dokumente aus der Kombination beider

### Schritt 2: Backend neu starten

Damit die Änderungen wirksam werden, musst du das Backend neu starten:

```bash
# Backend stoppen (Ctrl+C im Terminal)
# Backend neu starten
make backend
```

### Schritt 3: Evaluation durchführen

Führe die Evaluation aus, um die Retrieval-Qualität zu messen:

```bash
make evaluate
```

### Schritt 4: Vergleich in Langfuse

1. Öffne das Langfuse Dashboard: http://localhost:3000
2. Navigiere zu "Experiments" oder "Datasets"
3. Suche nach den neuesten Evaluation Runs:
   - Einer mit `retriever_SIMILARITY_SEARCH`
   - Einer mit `retriever_HYBRID_SEARCH`

### Wichtige Metriken zum Vergleichen

**Context Source Recall:**
- Misst, ob alle relevanten Dokumente gefunden wurden
- Höherer Wert = bessere Abdeckung

**Context Source Precision:**
- Misst, ob die gefundenen Dokumente relevant sind
- Höherer Wert = weniger irrelevante Dokumente

**Context Source F1:**
- Harmonisches Mittel aus Precision und Recall
- Beste Balance-Metrik

### Diskussionsfragen

1. **Welche Methode hat bessere Metriken?** Warum könnte das so sein?

2. **Gibt es spezifische Fragen**, bei denen Hybrid Search deutlich besser ist?
   - Schaue dir einzelne Traces in Langfuse an
   - Achte auf Fragen mit mehrdeutigen Begriffen oder spezifischen Eigennamen

3. **Performance-Überlegungen:**
   - BM25 benötigt alle Dokumente im Speicher
   - Vector Search skaliert besser bei sehr großen Datenmengen
   - Hybrid Search ist langsamer, da zwei Retriever ausgeführt werden

4. **Wann würdest du welche Methode wählen?**
   - Similarity: Allgemeine semantische Suche, große Datenmengen
   - Hybrid: Wenn exakte Begriffe UND semantisches Verständnis wichtig sind
   - BM25: Rechtsdokumente mit exakten Zitaten und Begriffen

5. **Das andrena-Problem in der echten Welt:**
   - Gibt es in deiner Domäne ähnliche mehrdeutige Begriffe?
   - Wie würde Hybrid Search dabei helfen?
   - Welche zusätzlichen Strategien (Query Expansion, Re-Ranking) könnten nützlich sein?